In [1]:
import json
import http.server
import socketserver
import requests
from http.server import HTTPServer
from http.server import BaseHTTPRequestHandler
from urllib.parse import urlparse
from saml2 import (
    BINDING_HTTP_POST,
    BINDING_HTTP_REDIRECT,
    entity,
)
from saml2.client import Saml2Client
from saml2.config import Config as Saml2Config
from urllib.parse import urlencode, quote_plus, parse_qs

PORT = 8000
DOMAIN="localhost"
URL="http://localhost:{0}".format (PORT)

def encode_url (url, payload):
    return "{0}/?{1}".format (url, urlencode(payload, quote_via=quote_plus))

payload = {'a':'b', 'c':'xyz'}

def url_for (path, _external=False, _scheme="http", **kwargs):
    prefix = ""
    if _external:
        prefix = "{0}://{1}:{2}".format (_scheme, DOMAIN, PORT)
    return encode_url ("{0}/{1}".format (prefix, path), kwargs)

print (url_for ("idp_initiated", _external=True, _scheme="https", **payload))

metadata_url_for = {
    "test" : "http://www.testshib.org/metadata/testshib-providers.xml"
    # For testing with http://saml.oktadev.com use the following:
    # 'test': 'http://idp.oktadev.com/metadata',
    # WARNING WARNING WARNING
    #   You MUST remove the testing IdP from a production system,
    #   as the testing IdP will allow ANYBODY to log in as ANY USER!
    # WARNING WARNING WARNING
}
class Request (object):
    def __init__(self, param, form):
        self.param = param
        self.form = form
        
def saml_client_for(idp_name=None):
    '''
    Given the name of an IdP, return a configuation.
    The configuration is a hash for use by saml2.config.Config
    '''

    if idp_name not in metadata_url_for:
        raise Exception("Settings for IDP '{}' not found".format(idp_name))
    acs_url = url_for(
        "idp_initiated",
        idp_name=idp_name,
        _external=True)
    https_acs_url = url_for(
        "idp_initiated",
        idp_name=idp_name,
        _external=True,
        _scheme='https')

    #   SAML metadata changes very rarely. On a production system,
    #   this data should be cached as approprate for your production system.
    rv = requests.get(metadata_url_for[idp_name])

    settings = {
        'metadata': {
            'inline': [rv.text],
            },
        'service': {
            'sp': {
                'endpoints': {
                    'assertion_consumer_service': [
                        (acs_url, BINDING_HTTP_REDIRECT),
                        (acs_url, BINDING_HTTP_POST),
                        (https_acs_url, BINDING_HTTP_REDIRECT),
                        (https_acs_url, BINDING_HTTP_POST)
                    ],
                },
                # Don't verify that the incoming requests originate from us via
                # the built-in cache for authn request ids in pysaml2
                'allow_unsolicited': True,
                # Don't sign authn requests, since signed requests only make
                # sense in a situation where you control both the SP and IdP
                'authn_requests_signed': False,
                'logout_requests_signed': True,
                'want_assertions_signed': True,
                'want_response_signed': False,
            },
        },
    }
    spConfig = Saml2Config()
    spConfig.load(settings)
    spConfig.allow_unknown_attributes = True
    saml_client = Saml2Client(config=spConfig)
    return saml_client

def idp_initiated_unc (request):
    return idp_initiated ('test', request)

#@app.route("/saml/sso/<idp_name>", methods=['POST'])
def idp_initiated(idp_name, request):
    saml_client = saml_client_for(idp_name)
    print (request.form['SAMLResponse'])
    authn_response = saml_client.parse_authn_request_response(
        request.form['SAMLResponse'],
        entity.BINDING_HTTP_POST)
    authn_response.get_identity()
    user_info = authn_response.get_subject()
    username = user_info.text
    
    print ('  username: {0}'.format (username))
    print ('first_name: {0}'.format (authn_response.ava['FirstName'][0]))
    print (' last_name: {0}'.format (authn_response.ava['LastName'][0]))

    session['saml_attributes'] = authn_response.ava
    #login_user(user) # ************** (flask)
    #url = url_for('user') # ****** (flask) ,,,,,
    # NOTE:
    #   On a production system, the RelayState MUST be checked
    #   to make sure it doesn't contain dangerous URLs!
    if 'RelayState' in request.form:
        url = request.form['RelayState']
    return url

#@app.route("/saml/login/<idp_name>")
def sp_initiated(idp_name):
    saml_client = saml_client_for (idp_name)
    reqid, info = saml_client.prepare_for_authenticate()

    redirect_url = None
    # Select the IdP URL to send the AuthN request to
    for key, value in info['headers']:
        if key is 'Location':
            redirect_url = value
    response = redirect(redirect_url, code=302)
    # NOTE:
    #   I realize I _technically_ don't need to set Cache-Control or Pragma:
    #     http://stackoverflow.com/a/5494469
    #   However, Section 3.2.3.2 of the SAML spec suggests they are set:
    #     http://docs.oasis-open.org/security/saml/v2.0/saml-bindings-2.0-os.pdf
    #   We set those headers here as a "belt and suspenders" approach,
    #   since enterprise environments don't always conform to RFCs
    response.headers['Cache-Control'] = 'no-cache, no-store'
    response.headers['Pragma'] = 'no-cache'
    return response

class AppHandler (BaseHTTPRequestHandler):
    def __init__(self, request, client_address, server):
        self.routes = {
            "/saml/sso/unc" : self.authenticate,
            "/act/"  : self.act
        }
        BaseHTTPRequestHandler.__init__(self, request, client_address, server)
    def reply (self, text, status=200):
        self.send_response (status)
        self.end_headers()
        self.wfile.write (text.encode ())
        #self.wfile.write ("\n")
    def redirect (self, url):
        self.send_header('Location', url)
        self.send_response (302)
        self.end_headers()
    def not_found (self, path):
        self.reply ("No such path: {0}".format (path), status=404)
    def authenticate (self, request):
        print ("{0}".format (request.form)) #json.dumps (request.form, indent=2))
        url = idp_initiated_unc (request)
        self.reply ("ok----------------------------------------- {}".format (url))
        #self.redirect (url)
    def act (self, request):
        self.reply ("                          action 097098798709870987809709790870987097978789")
    def saml (self, request):
        self.reply ("                          saml 77777777777777777")
    def do_POST(self):
        params = urlparse (self.path)
        path = params.path
        length = int(self.headers['Content-Length'])
        post_data = self.rfile.read(length)
        form= json.loads (post_data) # str(parse_qs(post_data)))
        print ("params: {0}".format (params)) 
        print ("form:   {0}".format (form)) #json.dumps ( json.loads (form), indent=2)))
        request = Request (param=params, form=form)
        if path in self.routes:
            self.routes[path](request)
        else:
            self.not_found (path)
        return


https://localhost:8000/idp_initiated/?a=b&c=xyz


In [ ]:
server = HTTPServer (('', PORT), AppHandler)
print ('Started http server on port ' , PORT)
while True:
    server.handle_request()
#server.serve_forever()

    

Started http server on port  8000
params: ParseResult(scheme='', netloc='', path='/saml/sso/unc', params='', query='', fragment='')
form:   {'SAMLResponse': '<samlp:Response\n    xmlns:samlp="urn:oasis:names:tc:SAML:2.0:protocol"\n    ID="Rf75c3375-1fa9-4f02-8f90-52c56479cde1"\n    Version="2.0"\n    IssueInstant="2012-01-25T17:57:29Z"\n    Destination="https://NoMoreClipboard.com">\n <saml:Issuer xmlns:saml="urn:oasis:names:tc:SAML:2.0:assertion">https://saml.example.com</saml:Issuer>\n <Signature xmlns="http://www.w3.org/2000/09/xmldsig#">\n  <SignedInfo>\n   <CanonicalizationMethod Algorithm="http://www.w3.org/2001/10/xml-exc-c14n#" />\n   <SignatureMethod Algorithm="http://www.w3.org/2000/09/xmldsig#rsa-sha1" />\n   <Reference URI="#Rf75c3375-1fa9-4f02-8f90-52c56479cde1">\n    <Transforms>\n     <Transform Algorithm="http://www.w3.org/2000/09/xmldsig#enveloped-signature" />\n     <Transform Algorithm="http://www.w3.org/2001/10/xml-exc-c14n#" />\n    </Transforms>\n    <DigestMethod

EXCEPTION: not well-formed (invalid token): line 1, column 0
Traceback (most recent call last):
  File "/Users/scox/dev/venv/trans/lib/python3.6/site-packages/saml2/response.py", line 337, in _loads
    **args)
  File "/Users/scox/dev/venv/trans/lib/python3.6/site-packages/saml2/sigver.py", line 1753, in correctly_signed_response
    response = samlp.any_response_from_string(decoded_xml)
  File "/Users/scox/dev/venv/trans/lib/python3.6/site-packages/saml2/samlp.py", line 1847, in any_response_from_string
    resp = func(xmlstr)
  File "/Users/scox/dev/venv/trans/lib/python3.6/site-packages/saml2/samlp.py", line 1446, in status_response_type__from_string
    return saml2.create_class_from_xml_string(StatusResponseType_, xml_string)
  File "/Users/scox/dev/venv/trans/lib/python3.6/site-packages/saml2/__init__.py", line 90, in create_class_from_xml_string
    tree = ElementTree.fromstring(xml_string)
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/pyth

<samlp:Response
    xmlns:samlp="urn:oasis:names:tc:SAML:2.0:protocol"
    ID="Rf75c3375-1fa9-4f02-8f90-52c56479cde1"
    Version="2.0"
    IssueInstant="2012-01-25T17:57:29Z"
    Destination="https://NoMoreClipboard.com">
 <saml:Issuer xmlns:saml="urn:oasis:names:tc:SAML:2.0:assertion">https://saml.example.com</saml:Issuer>
 <Signature xmlns="http://www.w3.org/2000/09/xmldsig#">
  <SignedInfo>
   <CanonicalizationMethod Algorithm="http://www.w3.org/2001/10/xml-exc-c14n#" />
   <SignatureMethod Algorithm="http://www.w3.org/2000/09/xmldsig#rsa-sha1" />
   <Reference URI="#Rf75c3375-1fa9-4f02-8f90-52c56479cde1">
    <Transforms>
     <Transform Algorithm="http://www.w3.org/2000/09/xmldsig#enveloped-signature" />
     <Transform Algorithm="http://www.w3.org/2001/10/xml-exc-c14n#" />
    </Transforms>
    <DigestMethod Algorithm="http://www.w3.org/2000/09/xmldsig#sha1" />
    <DigestValue>5zno9n7vQQIc6bnVbiaUM4272xk=</DigestValue>
   </Reference>
  </SignedInfo>
  <SignatureValue>LlHu/jHpI

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 56797)
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "<ipython-input-1-e593a6e0d9e1>", line 158, in __init__
    BaseHTTPRequestHandler.__init__(self, request, client_address, server)
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/socketserver.py", line 69